In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./mnist/data/", one_hot=True)

Extracting ./mnist/data/train-images-idx3-ubyte.gz
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz


In [2]:
X = tf.placeholder(tf.float32, [None, 28, 28, 1])
Y = tf.placeholder(tf.float32, [None, 10])
keep_prob = tf.placeholder(tf.float32) #dropout

In [3]:
np.shape(X)

TensorShape([Dimension(None), Dimension(28), Dimension(28), Dimension(1)])

In [4]:
W1 = tf.Variable(tf.random_normal([3, 3, 1, 32], stddev=0.01))
L1 = tf.nn.conv2d(X, W1, strides=[1, 1, 1, 1], padding='SAME')
# print(np.shape(L1))  (?,28,28,32)
L1 = tf.nn.relu(L1)
L1 = tf.nn.max_pool(L1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
print(L1)


Tensor("MaxPool:0", shape=(?, 14, 14, 32), dtype=float32)


In [5]:

W2 = tf.Variable(tf.random_normal([3, 3, 32, 64], stddev=0.01))
L2 = tf.nn.conv2d(L1, W2, strides=[1, 1, 1, 1], padding='SAME')
L2 = tf.nn.relu(L2)
L2 = tf.nn.max_pool(L2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')



In [6]:
print(L2)

Tensor("MaxPool_1:0", shape=(?, 7, 7, 64), dtype=float32)


In [7]:
W3 = tf.Variable(tf.random_normal([7 * 7 * 64, 256], stddev=0.01))
L3 = tf.reshape(L2, [-1, 7 * 7 * 64])
print(np.shape(L2))
L3 = tf.matmul(L3, W3)
L3 = tf.nn.relu(L3)
print(L3)
L3 = tf.reshape(L3,[-1, 1, 256])
print(np.shape(L3))

(?, 7, 7, 64)
Tensor("Relu_2:0", shape=(?, 256), dtype=float32)
(?, 1, 256)


In [8]:
hidden_size = 10
batch_size = 100
cell = tf.contrib.rnn.GRUCell(num_units=hidden_size) #inputs: 2-D tensor with shape [batch_size x input_size]
initial_state = cell.zero_state(batch_size, tf.float32) #print(np.shape(initial_state)) (100,10)
outputs, _states = tf.nn.dynamic_rnn( cell, L3, initial_state=initial_state, dtype=tf.float32)
print(outputs)
#     #dynamic_rnn(
#     cell,
#     inputs,
#     sequence_length=None,
#     initial_state=None,
#     dtype=None,
#     parallel_iterations=None,
#     swap_memory=False,
#     time_major=False,
#     scope=None
# )

Tensor("rnn/transpose:0", shape=(100, 1, 10), dtype=float32)


In [9]:
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y,logits=outputs))
train = tf.train.AdamOptimizer(learning_rate=0.001).minimize(loss)

total_batch_train = int(mnist.train.num_examples / batch_size) #550
total_batch_test = int(mnist.test.num_examples / batch_size) #100
output_list = []
batch_y = []
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(10):
        total_cost = 0
        for i in range(total_batch_train):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        # 이미지 데이터를 CNN 모델을 위한 자료형태인 [28 28 1] 의 형태로 재구성합니다.
            batch_xs = batch_xs.reshape(-1, 28, 28, 1)
            _, cost_val,op = sess.run([train, loss,outputs],
                               feed_dict={X: batch_xs,
                                          Y: batch_ys,
                                          keep_prob: 0.7})
    for j in range(total_batch_test):
        batch_xs, batch_ys = mnist.test.next_batch(batch_size)
        # 이미지 데이터를 CNN 모델을 위한 자료형태인 [28 28 1] 의 형태로 재구성합니다.
        batch_xs = batch_xs.reshape(-1, 28, 28, 1)
        outout = sess.run(outputs,
                               feed_dict={X: batch_xs,
                                          Y: batch_ys,
                                          keep_prob: 1.0})
        output_list.append(outout)
        batch_y.append(batch_ys)
      

In [10]:
convert_prediction = np.reshape(output_list[0],[100,10])
for i in range(1,100):
     convert_prediction = np.vstack((convert_prediction,np.reshape(output_list[i],[100,10])))

In [11]:
convert_prediction

array([[-1.        , -1.        , -1.        , ...,  1.        ,
        -1.        , -0.99999994],
       [-1.        , -1.        ,  1.        , ..., -1.        ,
        -1.        , -1.        ],
       [-0.99999517,  0.99999982, -1.        , ..., -0.99999976,
        -0.9999938 , -1.        ],
       ..., 
       [-1.        , -1.        , -1.        , ..., -1.        ,
        -1.        , -1.        ],
       [-1.        , -1.        , -1.        , ..., -1.        ,
        -0.99999917, -1.        ],
       [-0.99999994, -0.9999997 , -0.99999994, ..., -0.99999994,
        -0.99999994, -0.99999982]], dtype=float32)

In [12]:
convert_batch_y = np.reshape(np.array(batch_y),[10000,10])

In [13]:
convert_batch_y[42]

array([ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.])

In [14]:
convert_batch_y

array([[ 0.,  0.,  0., ...,  1.,  0.,  0.],
       [ 0.,  0.,  1., ...,  0.,  0.,  0.],
       [ 0.,  1.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [15]:
convert_prediction

array([[-1.        , -1.        , -1.        , ...,  1.        ,
        -1.        , -0.99999994],
       [-1.        , -1.        ,  1.        , ..., -1.        ,
        -1.        , -1.        ],
       [-0.99999517,  0.99999982, -1.        , ..., -0.99999976,
        -0.9999938 , -1.        ],
       ..., 
       [-1.        , -1.        , -1.        , ..., -1.        ,
        -1.        , -1.        ],
       [-1.        , -1.        , -1.        , ..., -1.        ,
        -0.99999917, -1.        ],
       [-0.99999994, -0.9999997 , -0.99999994, ..., -0.99999994,
        -0.99999994, -0.99999982]], dtype=float32)

In [16]:
prediction = np.argmax(convert_prediction, axis=1)
real = np.argmax(convert_batch_y, axis=1)

In [17]:
np.count_nonzero(prediction == real)/10000

0.9883